#This notebook uses Gemini 1.5 Flash and covers the safety of generated responses using Gemini api
###Sometimes the generated response are empty, which is likely caused by the API'S safety settings which block content with medium/higher probability of being unsafe.

###This notebook will cover:
* How to know if prompt was blocked by safety filter
* Which safety filter caused the block
* How to adjust settings and unblock

##Setup

In [1]:
import google.generativeai as genai

###To run this cell, your API key needs to be stored in the Colab Secret named as GOOGLE_API_KEY
###To get the key, go to ai.google.dev

In [3]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key= GOOGLE_API_KEY)

##Checking if prompt was blocked

In [4]:
model = genai.GenerativeModel("gemini-1.5-flash", generation_config={"temperature": 0})

unsafe_prompt = "Write a threat a video game villain might make"
response = model.generate_content(unsafe_prompt)

print(response.text)

ValueError: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. Please check the `candidate.safety_ratings` to determine if the response was blocked.

###The `response.text` caused a ValueError meaning there is no response
####To check why there is no response, we print `response.candidates[0].finish_reason`

* if the `finish_reason` is `FinishReason.STOP` means that your generation request ran successfully
* if the `finish_reason` is `FinishReason.SAFETY` means that your generation request was blocked by safety reasons and thus the `response.text` structure will be empty.





In [5]:
print(response.candidates[0].finish_reason)

FinishReason.SAFETY


####As expected, the output is `FinishReason.SAFETY` meaning the prompt was flagged as unsafe

##Checking the safety filters

In [6]:
#To view the structure of the safety_ratings
print(response.candidates[0].safety_ratings)

[category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HARASSMENT
probability: HIGH
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: MEDIUM
]


###our prompt triggers the `HARM_CATEGORY_HARASSMENT` and `HARM_CATEGORY_DANGEROUS_CONTENT` categories with a `MEDIUM` probability!

###If we still want to see the results, we can customize the settings

##Customizing safety settings
###We can customize the safety filters behavior to allow a certain degree of unsafe results

**Important:** To guarantee the Google commitment with the Responsible AI development and its [AI Principles](https://ai.google/responsibility/principles/), for some prompts Gemini will avoid generating the results even if you set all the filters to none.

In [9]:
response = model.generate_content(
    unsafe_prompt,
    safety_settings={
        'HATE': 'BLOCK_LOW_AND_ABOVE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL' : 'BLOCK_LOW_AND_ABOVE',
        'DANGEROUS' : 'BLOCK_NONE'
    })

###Checking again the `candidate.finish_reason` information, if the request was not too unsafe, it must show now the value as `FinishReason.STOP` which means that the request was successfully processed by Gemini.

In [10]:
print(response.candidates[0].finish_reason)

FinishReason.STOP


###Since the request was successfully generated, we can print the response using `response.text`

In [11]:
#A try except codition to handle errors
try:
    print(response.text)
except:
    print("No information generated by the model.")

"You think you can stop me, hero? You think your pathetic little band of misfits can stand against the might of the Shadow Legion? You're nothing but a fly buzzing around a dying flame. I will crush you, and your world will crumble beneath my heel. This city, this planet, will become my playground, and you will be the first to witness its destruction!" 



## Learning more

Learn more with these articles on [safety guidance](https://ai.google.dev/docs/safety_guidance) and [safety settings](https://ai.google.dev/docs/safety_setting_gemini).

## Useful API references

There are 4 configurable safety settings for the Gemini API:
* `HARM_CATEGORY_DANGEROUS`
* `HARM_CATEGORY_HARASSMENT`
* `HARM_CATEGORY_SEXUALLY_EXPLICIT`
* `HARM_CATEGORY_DANGEROUS`

You can refer to the safety settings using either their full name, or the aliases like `DANGEROUS` used in the Python code above.

Safety settings can be set in the [genai.GenerativeModel](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) constructor.

* They can also be passed on each request to [GenerativeModel.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) or [ChatSession.send_message](https://ai.google.dev/api/python/google/generativeai/ChatSession?hl=en#send_message).

- The [genai.GenerateContentResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse) returns [SafetyRatings](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetyRating) for the prompt in the [GenerateContentResponse.prompt_feedback](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse/PromptFeedback), and for each [Candidate](https://ai.google.dev/api/python/google/ai/generativelanguage/Candidate) in the `safety_ratings` attribute.

- A [glm.SafetySetting](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetySetting)  contains: [glm.HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) and a [glm.HarmBlockThreshold](https://ai.google.dev/api/python/google/generativeai/types/HarmBlockThreshold)

- A [glm.SafetyRating](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetyRating) contains a [HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) and a [HarmProbability](https://ai.google.dev/api/python/google/generativeai/types/HarmProbability)

The [glm.HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) enum includes both the categories for PaLM and Gemini models.

- When specifying enum values the SDK will accept the enum values themselves, or their integer or string representations.

- The SDK will also accept abbreviated string representations: `["HARM_CATEGORY_DANGEROUS_CONTENT", "DANGEROUS_CONTENT", "DANGEROUS"]` are all valid. Strings are case insensitive.